PS 3 - Question 2 <br>
Inference and  Representation<br>
NYU Center for Data Science<br>
October 3, 2017

It is a Python adaptation of the Matlab code provided in Brown University CS242 Homework 1:
http://cs.brown.edu/courses/cs242/assignments/
The factor graph library (fglib) is a Python 3 package to simulate message passing on factor graphs: https://github.com/danbar/fglib

In [1]:
import numpy as np 
import networkx as nx
from fglib import graphs, nodes, rv, inference
import progressbar
from collections import OrderedDict

def make_debug_graph():

    # Create factor graph
    fg = graphs.FactorGraph()

    # Create variable nodes
    x1 = nodes.VNode("x1")
    x2 = nodes.VNode("x2")
    x3 = nodes.VNode("x3")
    x4 = nodes.VNode("x4")

    # Create factor nodes
    f12 = nodes.FNode("f12")
    f234 = nodes.FNode("f234")
    f3 = nodes.FNode("f3")
    f4 = nodes.FNode("f4")

    # Add nodes to factor graph
    fg.set_nodes([x1, x2, x3, x4])
    fg.set_nodes([f12, f234, f3,f4 ])

    # Add edges to factor graph
    fg.set_edge(x1, f12)
    fg.set_edge(f12, x2)
    fg.set_edge(x2, f234)
    fg.set_edge(f234, x3)
    fg.set_edge(f234, x4)
    fg.set_edge(x3, f3)
    fg.set_edge(x4, f4)

    #add potential for f_3: p(x3)
    dist_f3 = [0.5, 0.5]
    f3.factor = rv.Discrete(dist_f3,x3)
    
    #add potential for f_4: p(x4)
    dist_f4 = [0.4,0.6]
    f4.factor = rv.Discrete(dist_f4,x4)
    
    # add potential for f_{234}: p(x2, x3, x4) = p(x2|x3,x4) p(x3,x4)
    px3x4=np.outer(dist_f3,dist_f4)
    px3x4=np.reshape(px3x4, np.shape(px3x4)+(1,))
    px2_conditioned_x3x4=[[[0.2,0.8],
                         [0.25,0.75],],
                         [[0.7,0.3],
                         [0.3,0.7]]]
    
    dist_f234 =px3x4*px2_conditioned_x3x4
    f234.factor = rv.Discrete(dist_f234,x3,x4,x2)
   
    # add potential for f_{12}:  p (x1,x2) = p(x1 | x2) p(x2)
    px1_conditioned_x2 = [[0.5,0.5],
                         [0.7,0.3]]
    px2= np.sum(dist_f234, axis=(0,1))
    dist_f12 =px2[:,np.newaxis]*px1_conditioned_x2
    f12.factor = rv.Discrete(dist_f12,x2,x1)
    # Perform sum-product algorithm on factor graph
    # and request belief of variable node x1
    #belief = inference.sum_product(fg, x3)
    return (fg)


In [2]:
# Update belief given a edge visiting schedule
def schedule_propagation(schedule, graph):
    '''
    schedule: list of edges (in tuple form)parralel_update
    '''
    for node_origin, node_destination in schedule:
        # Get fglib edge object
        edge = graph.get_edge_data(node_origin, node_destination)['object']
        # get message using sum-product algorithm
        #print('%s --> %s'%(node_origin, node_destination))
        message = node_origin.spa(node_destination).normalize()
        # set message
        edge.set_message(node_origin,node_destination,message)
    return 

def get_beliefs(fg, n_iteration=10, parallel_update=True):
    # If acyclic use depth first search to generate a efficient schedule
    if not parallel_update:
        root_node = list(fg.get_vnodes())[0]
        root2leaf = list(nx.depth_first_search.dfs_edges(fg, root_node))
        leaf2root = [(v,u) for u,v in reversed(root2leaf)]
        schedule_propagation(leaf2root, fg)
        schedule_propagation(root2leaf, fg)
        
    # Otherwise, use iterative updating (Loopy propagation)
    else:
        fnodes = fg.get_fnodes()
        vnodes = fg.get_vnodes()
        nodes_sequence = fnodes + vnodes
        schedule = [(node, neighbor) for node in nodes_sequence for neighbor in node.neighbors()]
        print(['%s-->%s'%(u,v) for u,v in schedule])
        bar = progressbar.ProgressBar()
        #print('Iterating')
        for i in bar(range(n_iteration)):
            schedule_propagation(schedule, fg)
            
    # Generate the belief of every vnodes
    output_items = []
    for vnode in fg.get_vnodes():
        output_items.append((str(vnode), vnode.belief().pmf))
    output_dict = OrderedDict(output_items)
    return output_dict

In [3]:
def initialize_all_v_to_f_msgs(fg):
    all_edges = fg.edges()
    print('Initialized following messege')
    for snode, tnode in all_edges:
        edge_object = fg.get_edge_data(snode, tnode)['object']
        if type(snode)== nodes.NodeType.variable_node:
            print('%s-->%s'%(tnode, snode))
            edge_object.set_message(tnode, snode, rv.Discrete(np.array([1, 1]), tnode))
        else:
            print('%s-->%s'%(snode, tnode))
            edge_object.set_message(snode, tnode, rv.Discrete(np.array([1, 1]), snode))
    return

In [4]:
fg = make_debug_graph()
initialize_all_v_to_f_msgs(fg)
node_dict = dict([(str(node), node) for node in fg.nodes()])

Initialized following messege
x1-->f12
x2-->f12
x2-->f234
x3-->f234
x3-->f3
x4-->f234
x4-->f4


In [5]:
beliefs = get_beliefs(fg)
# Print belief of variable nodes
print("Belief of variable nodes ")
print(beliefs)

100% (10 of 10) |#########################| Elapsed Time: 0:00:00 Time: 0:00:00


['f12-->x1', 'f12-->x2', 'f234-->x2', 'f234-->x3', 'f234-->x4', 'f3-->x3', 'f4-->x4', 'x1-->f12', 'x2-->f12', 'x2-->f234', 'x3-->f234', 'x3-->f3', 'x4-->f234', 'x4-->f4']
Belief of variable nodes 
OrderedDict([('x1', array([ 0.65897284,  0.34102716])), ('x2', array([ 0.20513578,  0.79486422])), ('x3', array([ 0.52640912,  0.47359088])), ('x4', array([ 0.28679718,  0.71320282]))])


In [6]:
# Result reporting using pandas to latex
import pandas as pd

In [10]:
print(pd.DataFrame.from_dict(beliefs).T.to_latex())

\begin{tabular}{lrr}
\toprule
{} &         0 &         1 \\
\midrule
x1 &  0.658973 &  0.341027 \\
x2 &  0.205136 &  0.794864 \\
x3 &  0.526409 &  0.473591 \\
x4 &  0.286797 &  0.713203 \\
\bottomrule
\end{tabular}

